In [ ]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import plot_tree
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib.gridspec import GridSpec
from sklearn.model_selection import GridSearchCV
from datetime import datetime

In [ ]:
df = pd.read_csv("VENDOR_MASTER.csv", sep=";")

In [ ]:
numeric_cols_to_fix = [
    'BDOs and BU SPLIT_AS ',
    'BDOs and BU SPLIT_EPS',
    'BDOs and BU SPLIT_SPS',
    'BDOs and BU SPLIT_S&C',
    'BDOs and BU SPLIT_Other',
    'TOTAL_Weight NSB Total',
    'TOTAL_Gross Margin',
    'ADVANCED SOLUTIONS_Weight NSB BU',
    'ADVANCED SOLUTIONS_Weight NSB Total',
    'ADVANCED SOLUTIONS_Gross Margin',
    'ENDPOINT SOLUTIONS_Weight NSB BU',
    'ENDPOINT SOLUTIONS_Weight NSB Total',
    'ENDPOINT SOLUTIONS_Gross Margin',
    'MAVERICK_Weight NSB BU',
    'MAVERICK_Weight NSB Total',
    'MAVERICK_Gross Margin',
    'GCC_Weight NSB BU',
    'GCC_Weight NSB Total',
    'GCC_Gross Margin',
    'SCORE'
]

for col in numeric_cols_to_fix:
    if df[col].dtype == 'O':
        df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

df[numeric_cols_to_fix].dtypes


In [ ]:
int_cols = df.select_dtypes(include=['int64']).columns

df[int_cols] = df[int_cols].astype('float64')

col = 'NET SALES BILLED_NSB/Creation'

df[col] = (
    df[col]
    .astype(str)              
    .str.replace(',', '.', regex=False)
    .replace('', '0')        
)

df[col] = pd.to_numeric(df[col], errors='coerce')
categorical_people = [
    'BDOs and BU SPLIT_Silvia',
    'BDOs and BU SPLIT_Arnau',
    'BDOs and BU SPLIT_Dani',
    'BDOs and BU SPLIT_BDOs'
]

for col in categorical_people:
    df[col] = df[col].astype('object')
    categorical_people = [
    'BDOs and BU SPLIT_Silvia',
    'BDOs and BU SPLIT_Arnau',
    'BDOs and BU SPLIT_Dani',
    'BDOs and BU SPLIT_BDOs'
]

for col in categorical_people:
    df[col] = df[col].astype('object')

pd.set_option('display.max_columns', None)
df.head(3)

df = df.drop(columns=['VENDORS'])

In [ ]:
y = df['SCORE']
X = df.drop(columns=['SCORE'])

num_cols = X.select_dtypes(include=['float64']).columns

for c in num_cols:
    X[f"{c}_scaled"] = (X[c] - X[c].mean()) / X[c].std()

X = X.drop(columns=num_cols)

X.info()


In [ ]:
cols_cat_nan = ["BDOs and BU SPLIT_Distribution Model", "BDOs and BU SPLIT_Status"]
df[cols_cat_nan] = df[cols_cat_nan].fillna("None")


In [ ]:
df["TOP VENDORS_TOP Vendors"]

In [ ]:
df["TOP VENDORS_LIST"] = df["TOP VENDORS_TOP Vendors"] \
    .astype(str) \
    .str.split(",") \
    .apply(lambda lst: [int(x) for x in lst if x.strip()!=''])


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

top_vendor_dummies = pd.DataFrame(
    mlb.fit_transform(df["TOP VENDORS_LIST"]),
    columns=[f"TOPV_{v}" for v in mlb.classes_],
    index=df.index
)

df = pd.concat([df.drop(columns=["TOP VENDORS_TOP Vendors", "TOP VENDORS_LIST"]), top_vendor_dummies], axis=1)


In [ ]:
df.columns

In [ ]:
df_pre = df.copy()

df_pre = pd.get_dummies(df_pre, drop_first=False)

y = df_pre["SCORE"]
X = df_pre.drop(columns=["SCORE"])

In [ ]:
df_pre.info()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

lasso_cv = LassoCV(cv=5, random_state=42, max_iter=10000).fit(X_train, y_train)

alpha_opt = lasso_cv.alpha_
coefs = lasso_cv.coef_

print("Alpha óptimo:", alpha_opt)
print("Número de características no nulas:", (coefs != 0).sum())

y_pred = lasso_cv.predict(X_test)
print("MSE en test:", mean_squared_error(y_test, y_pred))
print("R² en test:", r2_score(y_test, y_pred))

In [ ]:
print("X_train shape: {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_train shape: {}".format(y_train.shape))
print("y_test shape: {}".format(y_test.shape))

In [ ]:
rf = RandomForestRegressor(
    n_estimators=500,
    max_depth=None,
    random_state=42
)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("RANDOM FOREST RESULTS")
print("MSE:", mse_rf)
print("RMSE:", np.sqrt(mse_rf))
print("R²:", r2_rf)

In [ ]:
gbr = GradientBoostingRegressor(random_state=42)
gbr.fit(X_train, y_train)

y_pred_gb = gbr.predict(X_test)

mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

print("\nGRADIENT BOOSTING RESULTS")
print("MSE:", mse_gb)
print("RMSE:", np.sqrt(mse_gb))
print("R²:", r2_gb)

In [ ]:
#GB
importances_gb = gbr.feature_importances_
indices_gb = np.argsort(importances_gb)[::-1]
top_n = 20

plt.figure(figsize=(10, 6))
plt.barh(range(top_n), importances_gb[indices_gb[:top_n]][::-1])
plt.yticks(range(top_n), X_train.columns[indices_gb[:top_n]][::-1])
plt.title("Top 20 Feature Importances - Gradient Boosting")
plt.xlabel("Importance")
plt.tight_layout()
plt.show()

#RF
importances_rf = rf.feature_importances_
indices_rf = np.argsort(importances_rf)[::-1]

plt.figure(figsize=(10, 6))
plt.barh(range(top_n), importances_rf[indices_rf[:top_n]][::-1])
plt.yticks(range(top_n), X_train.columns[indices_rf[:top_n]][::-1])
plt.title("Top 20 Feature Importances - Random Forest")
plt.xlabel("Importance")
plt.tight_layout()
plt.show()

In [ ]:
top_n = 10

#Gradient Boosting
importances_gb = gbr.feature_importances_
indices_gb = np.argsort(importances_gb)[::-1]
top_gb_features = X_train.columns[indices_gb[:top_n]]
top_gb_values = importances_gb[indices_gb[:top_n]]

#Random Forest
importances_rf = rf.feature_importances_
indices_rf = np.argsort(importances_rf)[::-1]
top_rf_features = X_train.columns[indices_rf[:top_n]]
top_rf_values = importances_rf[indices_rf[:top_n]]

#mètriques del Gradient Boosting
mse_gb = mean_squared_error(y_test, gbr.predict(X_test))
rmse_gb = np.sqrt(mse_gb)
r2_gb = r2_score(y_test, gbr.predict(X_test))

#mètriques del Random Forest
mse_rf = mean_squared_error(y_test, rf.predict(X_test))
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, rf.predict(X_test))

all_top_features = list(set(top_gb_features).union(set(top_rf_features)))

colors = plt.cm.tab20(np.linspace(0, 1, len(all_top_features)))
color_map = {feat: colors[i] for i, feat in enumerate(all_top_features)}

fig = plt.figure(figsize=(18, 10))
gs = GridSpec(2, 2, height_ratios=[4, 1])

# Gràfic d’importància del Gradient Boosting
ax1 = fig.add_subplot(gs[0, 0])
ax1.barh(
    top_gb_features[::-1],
    top_gb_values[::-1],
    color=[color_map[f] for f in top_gb_features[::-1]]
)
ax1.set_title("Top 10 Variables més Importants – Gradient Boosting")
ax1.set_xlabel("Importància")

# Gràfic d’importància del Random Forest
ax2 = fig.add_subplot(gs[0, 1])
ax2.barh(
    top_rf_features[::-1],
    top_rf_values[::-1],
    color=[color_map[f] for f in top_rf_features[::-1]]
)
ax2.set_title("Top 10 Variables més Importants – Random Forest")
ax2.set_xlabel("Importància")

ax3 = fig.add_subplot(gs[1, 0])
ax3.axis("off")
ax3.text(
    0.01,
    0.5,
    f"MSE: {mse_gb:.4f}\nRMSE: {rmse_gb:.4f}\nR²: {r2_gb:.4f}",
    fontsize=12
)

ax4 = fig.add_subplot(gs[1, 1])
ax4.axis("off")
ax4.text(
    0.01,
    0.5,
    f"MSE: {mse_rf:.4f}\nRMSE: {rmse_rf:.4f}\nR²: {r2_rf:.4f}",
    fontsize=12
)

plt.tight_layout()
plt.show()


In [ ]:
top_gb_features = X_train.columns[indices_gb[:10]]

X_train_top10 = X_train[top_gb_features]
X_test_top10 = X_test[top_gb_features]



vlasso = LassoCV(cv=5, random_state=42, max_iter=10000)
vlasso.fit(X_train_top10, y_train)

y_pred_vlasso = vlasso.predict(X_test_top10)


gb_top10 = GradientBoostingRegressor(random_state=42)
gb_top10.fit(X_train_top10, y_train)

y_pred_gb_top10 = gb_top10.predict(X_test_top10)


rf_top10 = RandomForestRegressor(
    n_estimators=500,
    max_depth=None,
    random_state=42
)
rf_top10.fit(X_train_top10, y_train)

y_pred_rf_top10 = rf_top10.predict(X_test_top10)


def mostrar_metrics(nom, y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    print(f"Resultats de {nom}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}\n")

mostrar_metrics("VLasso", y_test, y_pred_vlasso)
mostrar_metrics("Gradient Boosting (top10)", y_test, y_pred_gb_top10)
mostrar_metrics("Random Forest (top10)", y_test, y_pred_rf_top10)


In [ ]:
gbr_base = GradientBoostingRegressor(random_state=42)

param_grid = {
    "n_estimators": [100, 300, 500],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [2, 3, 4],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

grid = GridSearchCV(
    estimator=gbr_base,
    param_grid=param_grid,
    scoring="r2",
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)

print("Millor combinació d’hiperparàmetres:")
print(grid.best_params_)

print("\nR² en train:", grid.best_score_)

gbr_opt = grid.best_estimator_

y_pred_opt = gbr_opt.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score
mse_opt = mean_squared_error(y_test, y_pred_opt)
rmse_opt = np.sqrt(mse_opt)
r2_opt = r2_score(y_test, y_pred_opt)

print("\nRESULTATS DEL MODEL OPTIMITZAT")
print("MSE:", mse_opt)
print("RMSE:", rmse_opt)
print("R²:", r2_opt)

In [ ]:
# TOP 20 MODEL ORIGINAL
importances_gb_orig = gbr.feature_importances_
indices_gb_orig = np.argsort(importances_gb_orig)[::-1]

top20_orig_features = X_train.columns[indices_gb_orig[:20]]
top20_orig_values = importances_gb_orig[indices_gb_orig[:20]]

print("TOP 20 VARIABLES – MODEL ORIGINAL\n")
for f, v in zip(top20_orig_features, top20_orig_values):
    print(f"{f}: {v}")

# TOP 20 MODEL OPTIMITZAT
importances_gb_opt = gbr_opt.feature_importances_
indices_gb_opt = np.argsort(importances_gb_opt)[::-1]

top20_opt_features = X_train.columns[indices_gb_opt[:20]]
top20_opt_values = importances_gb_opt[indices_gb_opt[:20]]

print("\nTOP 20 VARIABLES – MODEL OPTIMITZAT\n")
for f, v in zip(top20_opt_features, top20_opt_values):
    print(f"{f}: {v}")

# COMPARACIÓ ENTRE MODELS
set_orig = set(top20_orig_features)
set_opt = set(top20_opt_features)

comunes = set_orig.intersection(set_opt)
nomes_original = set_orig - set_opt
nomes_optimitzat = set_opt - set_orig

print("\nVARIABLES COMUNES ALS DOS MODELS:\n")
for x in comunes:
    print(x)

print("\nVARIABLES QUE NOMÉS SURTEN AL MODEL ORIGINAL:\n")
for x in nomes_original:
    print(x)

print("\nVARIABLES QUE NOMÉS SURTEN AL MODEL OPTIMITZAT:\n")
for x in nomes_optimitzat:
    print(x)

In [ ]:
importances = gbr_opt.feature_importances_
indices = np.argsort(importances)[::-1]

top10_features = X_train.columns[indices[:10]]
top10_importances = importances[indices[:10]]

pesos_normalitzats = top10_importances / top10_importances.sum()

df_pesos = pd.DataFrame({
    "variable": top10_features,
    "importancia": top10_importances,
    "pes": pesos_normalitzats
}).sort_values(by="pes", ascending=False)

print("Pesos normalitzats de les top 10 variables:")
display(df_pesos)

def calcular_score_ponderat(row):
    return sum(row[var] * pes for var, pes in zip(top10_features, pesos_normalitzats))

X_train_score = X_train[top10_features].copy()
X_test_score = X_test[top10_features].copy()

X_train_score["score_ponderat"] = X_train_score.apply(calcular_score_ponderat, axis=1)
X_test_score["score_ponderat"] = X_test_score.apply(calcular_score_ponderat, axis=1)


In [ ]:
list(zip(X_train.columns, gbr_opt.feature_importances_))


In [ ]:
# 1. PESOS ORIGINALS DEFINITS DEL MODEL DE NEGOCI

pesos_originals = {
    "Creations 2025": 0.1,
    "Creations TOTAL": 0.1,
    "NSB H1 2025": 0.085,
    "NSB 24/25": 0.085,
    "Priority": 0.3,
    "BDO Priority": 0.25,
    "Distribution Model": 0.08
}

# 2. AGAFEM AUTOMÀTICAMENT ELS PESOS ML DEL TEU df_pesos

pesos_ml_full = dict(zip(df_pesos["variable"], df_pesos["pes"]))

# 3. CORRESPONDÈNCIES AUTOMÀTIQUES ORIGINAL ↔ ML

def pes(var): 
    return pesos_ml_full.get(var, 0)

pesos_ml_equivalents = {
    "Creations 2025": pes("CREATIONS_Creations 2025"),
    "Creations TOTAL": pes("CREATIONS_TOTAL"),
    "NSB H1 2025": pes("NET SALES BILLED_NSB (H1 2025)"),
    "NSB 24/25": pes("NET SALES BILLED_Total Sales (24/25?)"),
    "Priority": pes("7 VENDORS_Priority"),

    "BDO Priority": (
        pes("BDOs and BU SPLIT_BDOs_1.0") +
        pes("BDOs and BU SPLIT_BDOs_2.0") +
        pes("BDOs and BU SPLIT_BDOs_4.0")
    ),

    "Distribution Model": sum(
        val for key, val in pesos_ml_full.items()
        if "BDOs and BU SPLIT_Distribution Model" in key
    )
}

# 4. NORMALITZEM ELS PESOS ML PERQUÈ SUMIN 1

suma = sum(pesos_ml_equivalents.values())
pesos_ml_norm = {k: v / suma for k, v in pesos_ml_equivalents.items()}

# 5. DATAFRAME COMPARATIU

df_comp = pd.DataFrame({
    "pes_original": pesos_originals,
    "pes_ml": pesos_ml_norm
})

display(df_comp)

# 6. GRÀFIC COMPARATIU

df_comp.plot(kind="bar", figsize=(12, 6))
plt.title("Comparació de pesos: Model original vs Model ML (Top 10)")
plt.ylabel("Pes normalitzat")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.show()


In [ ]:
# 1) Importàncies globals des del model ML
llista = list(zip(X_train.columns, gbr_opt.feature_importances_))
df_full = pd.DataFrame(llista, columns=["variable", "importancia"])
df_full["pes_global"] = df_full["importancia"] / df_full["importancia"].sum()

# 2) Pesos originals (ja normalitzats)
pesos_originals = {
    "Creations 2025": 0.1,
    "Creations TOTAL": 0.1,
    "NSB H1 2025": 0.085,
    "NSB 24/25": 0.085,
    "Priority": 0.3,
    "BDO Priority": 0.25,
    "Distribution Model": 0.08
}

# 3) Correspondències ML -> variables originals
corresp = {
    "Creations 2025": ["CREATIONS_Creations 2025"],
    "Creations TOTAL": ["CREATIONS_TOTAL"],
    "NSB H1 2025": ["NET SALES BILLED_NSB (H1 2025)"],
    "NSB 24/25": ["NET SALES BILLED_Total Sales (24/25?)"],
    "Priority": ["7 VENDORS_Priority"],
    "BDO Priority": [
        "BDOs and BU SPLIT_BDOs_1.0",
        "BDOs and BU SPLIT_BDOs_2.0",
        "BDOs and BU SPLIT_BDOs_4.0"
    ],
    "Distribution Model": [
        var for var in df_full["variable"]
        if "BDOs and BU SPLIT_Distribution Model" in var
    ]
}

# 4) Pesos globals ML SUMATS per correspondència
pesos_ml_globals_raw = {
    nom: df_full.loc[df_full["variable"].isin(vars_ml), "pes_global"].sum()
    for nom, vars_ml in corresp.items()
}

# 5) Normalització correcta (només per als 7 pesos originals)
suma_globals = sum(pesos_ml_globals_raw.values())
pesos_ml_globals = {k: v / suma_globals for k, v in pesos_ml_globals_raw.items()}

# 6) DataFrame final comparatiu
df_global_comp = pd.DataFrame({
    "pes_original": pesos_originals,
    "pes_ml_global": pesos_ml_globals
})

display(df_global_comp)

# 7) Gràfic comparatiu
df_global_comp.plot(kind="bar", figsize=(12,6))
plt.title("Comparació de pesos: Model Original vs Pesos ML Globals (normalitzats)")
plt.ylabel("Pes normalitzat")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.show()

In [ ]:
pesos_originals = {
    "Creations 2025": 0.1,
    "Creations TOTAL": 0.1,
    "NSB H1 2025": 0.085,
    "NSB 24/25": 0.085,
    "Priority": 0.3,
    "BDO Priority": 0.25,
    "Distribution Model": 0.08
}

pesos_ml_top10 = dict(zip(df_pesos["variable"], df_pesos["pes"]))
def pes_top10(v): return pesos_ml_top10.get(v, 0)

pesos_top10_equivalents = {
    "Creations 2025": pes_top10("CREATIONS_Creations 2025"),
    "Creations TOTAL": pes_top10("CREATIONS_TOTAL"),
    "NSB H1 2025": pes_top10("NET SALES BILLED_NSB (H1 2025)"),
    "NSB 24/25": pes_top10("NET SALES BILLED_Total Sales (24/25?)"),
    "Priority": pes_top10("7 VENDORS_Priority"),
    "BDO Priority": (
        pes_top10("BDOs and BU SPLIT_BDOs_1.0") +
        pes_top10("BDOs and BU SPLIT_BDOs_2.0") +
        pes_top10("BDOs and BU SPLIT_BDOs_4.0")
    ),
    "Distribution Model": 0
}

s_top10 = sum(pesos_top10_equivalents.values())
pesos_top10_norm = {k: v / s_top10 for k, v in pesos_top10_equivalents.items()}

llista = list(zip(X_train.columns, gbr_opt.feature_importances_))
df_full = pd.DataFrame(llista, columns=["variable", "importancia"])
df_full["pes_global"] = df_full["importancia"] / df_full["importancia"].sum()

corresp = {
    "Creations 2025": ["CREATIONS_Creations 2025"],
    "Creations TOTAL": ["CREATIONS_TOTAL"],
    "NSB H1 2025": ["NET SALES BILLED_NSB (H1 2025)"],
    "NSB 24/25": ["NET SALES BILLED_Total Sales (24/25?)"],
    "Priority": ["7 VENDORS_Priority"],
    "BDO Priority": [
        "BDOs and BU SPLIT_BDOs_1.0",
        "BDOs and BU SPLIT_BDOs_2.0",
        "BDOs and BU SPLIT_BDOs_4.0"
    ],
    "Distribution Model": [
        v for v in df_full["variable"]
        if "BDOs and BU SPLIT_Distribution Model" in v
    ]
}

pesos_ml_global_raw = {
    nom: df_full.loc[df_full["variable"].isin(vars_ml), "pes_global"].sum()
    for nom, vars_ml in corresp.items()
}

s_global = sum(pesos_ml_global_raw.values())
pesos_ml_global = {k: v / s_global for k, v in pesos_ml_global_raw.items()}

df_triple = pd.DataFrame({
    "Original": pesos_originals,
    "Top10 ML": pesos_top10_norm,
    "Global ML": pesos_ml_global
})

display(df_triple)

df_triple.plot(kind="bar", figsize=(14,6))
plt.xticks(rotation=45)
plt.ylabel("Pes")
plt.title("Comparació de pesos: Original vs ML Top-10 vs ML Global")
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.show()
